# Evaluation Study

## Setup

In [3]:
import os, sys
import shutil
import math
import time
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.nn.functional as F

sys.path.append('..')

In [10]:
# PATH Setting
DIR_PATH = 'F:/20210730_samples'
SAVE_PATH = os.path.join(DIR_PATH, 'numpy')
CT_PATH = os.path.join(SAVE_PATH, 'CT_target')
PT_PATH = os.path.join(SAVE_PATH, 'PT_vanilla')
PT_PATH_2 = os.path.join(SAVE_PATH, 'PT_resize')

In [11]:
device = torch.device('cuda')

## NCC

In [117]:
Ii = np.zeros([16, 1, 128, 128, 256])
Ji = np.zeros([16, 1, 128, 128, 256])
for i in range(16):
    Ii[i,0] = np.load(os.path.join(CT_PATH, os.listdir(CT_PATH)[i]))
    Ji[i,0] = np.load(os.path.join(PT_PATH, os.listdir(PT_PATH)[i]))

Ii = torch.FloatTensor(Ii).to(device)
Ji = torch.FloatTensor(Ji).to(device)

In [119]:
Ii = np.zeros([16, 1, 128, 128, 256])
Ji = np.zeros([16, 1, 128, 128, 256])
for i in range(16):
    Ii[i,0] = np.load(os.path.join(CT_PATH, os.listdir(CT_PATH)[i]))
    Ji[i,0] = np.load(os.path.join(PT_PATH_2, os.listdir(PT_PATH_2)[i]))

Ii = torch.FloatTensor(Ii).to(device)
Ji = torch.FloatTensor(Ji).to(device)

In [19]:
import utils.model.losses as losses

start = time.time()

criterion = losses.NCC(local=False, metric=True, device=device).to(device)
cc = criterion(Ii, Ji).cpu()
print ('Autograd Check', cc.requires_grad)
print ('NCC', cc)
print ('Average NCC', cc.mean().item())
print ('Time:', time.time()-start)

Autograd Check False
NCC tensor([0.4268, 0.4540, 0.5476, 0.4465, 0.3494, 0.5016, 0.3968, 0.2531, 0.2562,
        0.2873, 0.4179, 0.3197, 0.3976, 0.3193, 0.5823, 0.3773])
Average NCC 0.3958446681499481
Time: 4.13455867767334


In [6]:
torch.linspace(4,5,10)

tensor([4.0000, 4.1111, 4.2222, 4.3333, 4.4444, 4.5556, 4.6667, 4.7778, 4.8889,
        5.0000])

In [123]:
import utils.model.losses as losses

start = time.time()

criterion = losses.MutualInformation(sigma_ratio=0.1, device=device).to(device)
cc = criterion(Ii[:4], Ii[:4]).cpu()
print ('Autograd Check', cc.requires_grad)
print ('NCC', cc)
print ('Average NCC', cc.mean().item())
print ('Time:', time.time()-start)

Autograd Check False
NCC tensor([0.9143, 0.8923, 0.9299, 0.9514])
Average NCC 0.9219973087310791
Time: 0.4689037799835205


In [31]:
a = np.array([[0.1, 0.2, 0.1],
              [0.9, 0.8, 0.9]])
b = np.array([[0.2, 0.3, 0.5],
              [0.8, 0.7, 0.5]])
np.sum(np.matmul(a, b.T))

3.0

In [76]:
a = np.arange(12).reshape(2,3,2)
a = torch.FloatTensor(a)
at = a / torch.sum(a, dim=-1, keepdims=True)
b = np.arange(12).reshape(2,3,2)
b = torch.FloatTensor(b)
bt = b / torch.sum(b, dim=-1, keepdims=True)
print (at,bt,sep='\n')

tensor([[[0.0000, 1.0000],
         [0.4000, 0.6000],
         [0.4444, 0.5556]],

        [[0.4615, 0.5385],
         [0.4706, 0.5294],
         [0.4762, 0.5238]]])
tensor([[[0.0000, 1.0000],
         [0.4000, 0.6000],
         [0.4444, 0.5556]],

        [[0.4615, 0.5385],
         [0.4706, 0.5294],
         [0.4762, 0.5238]]])


In [84]:
pa = torch.mean(at, dim=1, keepdim=True)
pb = torch.mean(bt, dim=1, keepdim=True)
papb = torch.bmm(pa.permute(0,2,1), pb)
papb

tensor([[[0.0792, 0.2022],
         [0.2022, 0.5163]],

        [[0.2204, 0.2491],
         [0.2491, 0.2815]]])

In [85]:
print (pa)
print (pb)

tensor([[[0.2815, 0.7185]],

        [[0.4694, 0.5306]]])
tensor([[[0.2815, 0.7185]],

        [[0.4694, 0.5306]]])


In [86]:
pab = torch.bmm(at.permute(0,2,1), bt)/3
pab

tensor([[[0.1192, 0.1623],
         [0.1623, 0.5562]],

        [[0.2204, 0.2490],
         [0.2490, 0.2815]]])

In [87]:
mi = torch.sum(torch.sum(pab * torch.log(pab / papb), dim=1), dim=1)

In [88]:
mi

tensor([ 1.8681e-02, -7.6761e-10])

In [89]:
torch.log(pab / papb)

tensor([[[ 4.0823e-01, -2.2003e-01],
         [-2.2003e-01,  7.4525e-02]],

        [[ 1.6533e-04, -1.4634e-04],
         [-1.4634e-04,  1.2945e-04]]])

In [90]:
torch.sum(pab * torch.log(pab / papb), dim=1)

tensor([[ 1.2940e-02,  5.7404e-03],
        [-2.8049e-09,  2.0373e-09]])

In [91]:
a = np.arange(6).reshape(3,2)
a = torch.FloatTensor(a)
at = a / torch.sum(a, dim=-1, keepdims=True)
b = np.arange(6).reshape(3,2)
b = torch.FloatTensor(b)
bt = b / torch.sum(b, dim=-1, keepdims=True)
print (at,bt,sep='\n')

tensor([[0.0000, 1.0000],
        [0.4000, 0.6000],
        [0.4444, 0.5556]])
tensor([[0.0000, 1.0000],
        [0.4000, 0.6000],
        [0.4444, 0.5556]])


In [95]:
pa = torch.mean(at, dim=0, keepdim=True)
pb = torch.mean(bt, dim=0, keepdim=True)
papb = torch.matmul(pa.permute(1,0), pb)
print (pa)
print (pb)
print (papb)

tensor([[0.2815, 0.7185]])
tensor([[0.2815, 0.7185]])
tensor([[0.0792, 0.2022],
        [0.2022, 0.5163]])


In [97]:
pab = torch.matmul(at.permute(1,0), bt)/3
pab

tensor([[0.1192, 0.1623],
        [0.1623, 0.5562]])

In [104]:
pab * torch.log2(pab / papb)

tensor([[ 0.0702, -0.0515],
        [-0.0515,  0.0598]])

In [102]:
 torch.sum(pab * torch.log2(pab / papb))

tensor(0.0270)

In [107]:
hx = -torch.sum(pb*torch.log2(pb))

In [106]:
hxy = -torch.sum(pab*torch.log2(pab))

In [108]:
2*hx-hxy

tensor(0.0270)

In [109]:
hxy

tensor(1.6880)

In [114]:
hx

tensor(0.8575)

In [4]:
a = np.arange(6).reshape(3,2)
a = torch.FloatTensor(a)
at = a / torch.sum(a, dim=-1, keepdims=True)
b = np.arange(6).reshape(3,2)
b = torch.FloatTensor(b)
bt = b / torch.sum(b, dim=-1, keepdims=True)
print (at,bt,sep='\n')

tensor([[0.0000, 1.0000],
        [0.4000, 0.6000],
        [0.4444, 0.5556]])
tensor([[0.0000, 1.0000],
        [0.4000, 0.6000],
        [0.4444, 0.5556]])


In [5]:
at

tensor([[0.0000, 1.0000],
        [0.4000, 0.6000],
        [0.4444, 0.5556]])

In [7]:
torch.argmin(at, dim=1)

tensor([0, 0, 0])

In [31]:
at = 0

In [32]:
at[[i for i in range(3)],[0,0,0]] = 1

TypeError: 'int' object does not support item assignment

In [30]:
at

tensor([[1.0000, 1.0000],
        [1.0000, 0.6000],
        [1.0000, 0.5556]])

In [33]:
a = np.arange(12).reshape(2,3,2)
a = torch.FloatTensor(a)
at = a / torch.sum(a, dim=-1, keepdims=True)
b = np.arange(12).reshape(2,3,2)
b = torch.FloatTensor(b)
bt = b / torch.sum(b, dim=-1, keepdims=True)
print (at,bt,sep='\n')

tensor([[[0.0000, 1.0000],
         [0.4000, 0.6000],
         [0.4444, 0.5556]],

        [[0.4615, 0.5385],
         [0.4706, 0.5294],
         [0.4762, 0.5238]]])
tensor([[[0.0000, 1.0000],
         [0.4000, 0.6000],
         [0.4444, 0.5556]],

        [[0.4615, 0.5385],
         [0.4706, 0.5294],
         [0.4762, 0.5238]]])


In [44]:
a

tensor([[0, 0, 0],
        [0, 0, 0]])

In [45]:
emp = torch.zeros_like(at)
emp

tensor([[[0., 0.],
         [0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.],
         [0., 0.]]])

In [48]:
emp[[0,0]]

tensor([[[0., 0.],
         [0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.],
         [0., 0.]]])

In [41]:
torch.max(at,dim=2)

torch.return_types.max(
values=tensor([[1.0000, 0.6000, 0.5556],
        [0.5385, 0.5294, 0.5238]]),
indices=tensor([[1, 1, 1],
        [1, 1, 1]]))